# Treinamento Python - pt1

<br>

Este notebook é a junção de outros códigos feitos para um grupo de estudos em Python. Para o treinamento, falaremos dos seguintes pontos:

* Funções built-in úteis
    * enumerate, zip, try except
    * comprehensions
* Pacotes para conectar a servidores de dados
    * sqlalchemy 
    * pyodbc
* Manipulação variáveis com DataFrames
    * Métodos prontos - vetorização 
    * map, apply, applymap
    * case when 
    * juntar categorias
    * criar dummies
    * .str e .dt
* Múltiplas bases
    * Cuidado com índices
    * pd.concat
    * pd.merge, pd.merge_asof

In [1]:
import os
import re
import numpy as np
import pandas as pd
from itertools import product

from sqlalchemy import create_engine
from pyodbc import connect
from getpass import getpass

from teamcapcodes.utils import *

# Funções built-in

## map, filter

* ``map(funcao, sequencia)``: aplica 'funcao' a cada elemento de 'sequencia'. Retorna uma sequencia 
* ``filter(funcao, sequencia)``: retorna os valores de 'sequencia' que satisfazem a condição booleana em 'funcao'

In [17]:
# map

score = [0.2, 0.8]
classifica_score = lambda x: 'pedido_mau' if x >= 0.5 else 'pedido_bom'
res = list(map(classifica_score, score))

print('\nPara os scores {}, temos as seguintes decisões: {}'.format(score, res))


Para os scores [0.2, 0.8], temos as seguintes decisões: ['pedido_bom', 'pedido_mau']


In [20]:
# filter

x = range(10)

list(filter(lambda x: x < 4, x))

[0, 1, 2, 3]

In [22]:
# outro uso do filter:

x = ['', 'nao vazio', None, 'nao vazio', 1, 2]

list(filter(None, x))



['nao vazio', 'nao vazio', 1, 2]

## loops com listas e range

In [2]:
a = list('abcd')
b = range(1,5)

print('Lista a: {}'.format(a))
print('Lista b: {}'.format(b))

Lista a: ['a', 'b', 'c', 'd']
Lista b: range(1, 5)


In [4]:
for j in a:
    print(j.upper())

A
B
C
D


In [12]:
for j in b:
    print(j ** 2)

1
4
9
16


## enumerate 

* Serve para iterar sobre um elemento x retendo não só os seus valores, como o índice relativo a esse valor. Note que deve usar, portanto, 2 variáveis dentro do 'for'

In [3]:
for ind, elem in enumerate(a):
    print('\nO índice do elemento "{}" é {}'.format(elem, ind))


O índice do elemento "a" é 0

O índice do elemento "b" é 1

O índice do elemento "c" é 2

O índice do elemento "d" é 3


In [11]:
for ind, elem in enumerate(a, start=1):
    print('\nO índice do elemento "{}" também pode ser {}'.format(elem, ind))


O índice do elemento "a" também pode ser 1

O índice do elemento "b" também pode ser 2

O índice do elemento "c" também pode ser 3

O índice do elemento "d" também pode ser 4


## zip

* A função zip é utilizada para juntar/concatenar duas ou mais listas/tuplas __de mesmo tamanho__
* Seu uso mais intuitivo é dentro de loops para poder iterar sobre 2 ou mais variáveis ao mesmo tempo
* Pode ser utilizada também para criar dicionários de maneira programática. 

In [4]:
for i, j in zip(a,b):
    print('\nIterando em "{}" e "{}"'.format(i,j))


Iterando em "a" e "1"

Iterando em "b" e "2"

Iterando em "c" e "3"

Iterando em "d" e "4"


In [6]:
list(zip(a[:2],b))

[('a', 1), ('b', 2)]

In [7]:
from itertools import product

In [8]:
list(product(a[:2],b))

[('a', 1),
 ('a', 2),
 ('a', 3),
 ('a', 4),
 ('b', 1),
 ('b', 2),
 ('b', 3),
 ('b', 4)]

In [22]:
# Criando dicionários mais facilmente:

print('\nUtilizando a lista "a" como chaves: {}'.format(dict(zip(a,b))))
print('\nUtilizando a lista "b" como chaves: {}'.format(dict(zip(b,a))))


Utilizando a lista "a" como chaves: {'a': 1, 'b': 2, 'c': 3, 'd': 4}

Utilizando a lista "b" como chaves: {1: 'a', 2: 'b', 3: 'c', 4: 'd'}


## try, except

Funções úteis para controle de outputs de loops ou funções em casos de erro

In [9]:
# Por exemplo, já tentou fazer operações com None?
# Se estiver trabalhando com uma lista que pode conter None, é interessante controlar o outpupt:

x = [1,2,3,None,5]

list(map(lambda x: x > 4, x))

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [10]:
# Solução:

def f(x):
    try:
        return x > 4
    except:
        return 'nao sei'
        
list(map(f, x))

[False, False, False, 'nao sei', True]

In [11]:
# É possível ser mais específico ainda, colocando excepts para cada tipo de erro:

x = [1,2,1,0,4,None]

def f(x):
    try:
        output = 1 / x
    except ZeroDivisionError:
        output = 1 / (x + 0.000001)
    except TypeError:
        output = 0
    return output
    
list(map(f, x))

[1.0, 0.5, 1.0, 1000000.0, 0.25, 0]

## Comprehensions

* Nada mais são do que sintaxes enxutas de for loops, em geral com melhor ou igual eficiência
* O output pode ser uma lista, um dicionário ou um generator
* Sintaxes para cada caso: 
    + list: __[f(x) for x in iterable]__
    + dict: __{k(x): v(x) for x in iterable}__
    + generator: __(f(x) for x in iterable)__

### list comprehension

In [12]:
# Objetivo: quebrar string com delimitador ' ' e depois remover vírgulas dos trechos

texto = ("Ele não detalhou o porquê "
         "de, na visão dele, o atual modelo de ensino gerar dependência, "
         "de programas sociais.")

print(texto)

Ele não detalhou o porquê de, na visão dele, o atual modelo de ensino gerar dependência, de programas sociais.


In [13]:
t = texto.split()

In [68]:
[s.replace(',','').replace('.','') for s in texto.split()]

['Ele',
 'não',
 'detalhou',
 'o',
 'porquê',
 'de',
 'na',
 'visão',
 'dele',
 'o',
 'atual',
 'modelo',
 'de',
 'ensino',
 'gerar',
 'dependência',
 'de',
 'programas',
 'sociais']

In [69]:
x = range(10,25,5)
[i ** 3 for i in x]

[1000, 3375, 8000]

In [79]:
# Introduzindo if-else statements

# 1) julgando todos os elementos da lista

x = list('aefgkl')

res = ['> c' if s > 'c' else '<= c' for s in x]

print("Caso 1: {}".format(res))

# 2) Filtrando elementos caso não correspondam a cláusula

res = [s for s in x if s > 'c']

print("Caso 2: {}".format(res))

Caso 1: ['<= c', '> c', '> c', '> c', '> c', '> c']
Caso 2: ['e', 'f', 'g', 'k', 'l']


### dict comprehension

In [81]:
# Criando dicionários com chaves sendo ordem índice da ordem alfabética

x = ['melancia','banana','maçã']

{val: ind for ind, val in enumerate(sorted(x),1)}

{'banana': 1, 'maçã': 2, 'melancia': 3}

### generators

In [15]:
# Iguais às list comprehensions, mas com lazy evaluation

x = range(10,25,5)

gen = (i ** 3 for i in x)
arqs = ['df1.csv','df2.csv']
gen = (pd.read_csv(i).assign(var=i) for i in arqs)
gen

<generator object <genexpr> at 0x00000202464E4ED0>

In [16]:
list(gen)

[1000, 3375, 8000]

# Conexão a bancos

In [17]:
def con_sqlalchemy(uid, pwd, server='172.27.16.212:4444', database='est_estudo'):
    return create_engine(f'mssql+pyodbc://{uid}:{pwd}@{server}/{database}?driver=SQL+Server')

In [18]:
def con_pyodbc(uid, pwd, server = '172.27.16.212,4444', database = 'est_estudo'):
    conn = connect('Driver={SQL Server};'
                   'Server='+server+';'
                   'Database='+database+';'
                   'uid='+uid+';pwd='+pwd)
    return conn

## pyodbc

In [19]:
con = con_pyodbc('eric.barbosa', getpass())

df = pd.read_sql('''
select top 10 
    * 
from clearsale..pedidos
''', con)

df.head(1)

con.close()

········


## sqlalchemy

In [25]:
con = con_sqlalchemy('eric.barbosa',getpass())

········


In [26]:
df = pd.read_sql('''
select top 10 
    * 
from clearsale..pedidos
''', con)

In [ ]:
# session do sqlalchemy

# Manipulação de variáveis

## Lendo tabela com histórico de pedidos por cpf

In [4]:
con = con_sqlalchemy('eric.barbosa', getpass())

········


In [27]:
query = '''
    select top 100000
        *
    from eric_barbosa.modelagem_contas_v3_gc_pedidos_cpf
'''

df = pd.read_sql(query, con)

In [28]:
print(df.info())
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
id                    100000 non-null float64
base                  100000 non-null object
id_registro           100000 non-null datetime64[ns]
pedido_id             100000 non-null float64
pedido_registro       100000 non-null datetime64[ns]
pedido_status_id      100000 non-null float64
entidade_id           100000 non-null float64
entidade_mundofisi    100000 non-null int64
entidade_santander    100000 non-null int64
entidade_emissores    100000 non-null int64
dtypes: datetime64[ns](2), float64(4), int64(3), object(1)
memory usage: 7.6+ MB
None


,id,base,id_registro,pedido_id,pedido_registro,pedido_status_id,entidade_id,entidade_mundofisi,entidade_santander,entidade_emissores
0,729015091.0,09449309485,2018-01-01 02:32:37.230,42329990.0,2011-08-02 20:02:42.070,18.0,100957.0,0,0,0
1,729015091.0,09449309485,2018-01-01 02:32:37.230,43606089.0,2011-08-15 19:59:36.960,26.0,101831.0,0,0,0


## Criando variáveis

In [29]:
# Verificando quais as colunas existentes:
list(df.columns)

['id',
 'base',
 'id_registro',
 'pedido_id',
 'pedido_registro',
 'pedido_status_id',
 'entidade_id',
 'entidade_mundofisi',
 'entidade_santander',
 'entidade_emissores']

In [30]:
# Todos os operadores matemáticos entre Series são vetorizados (se os tipos das colunas forem compatíveis)

print(df['pedido_id'].head(2) - 1000)

(df['entidade_emissores'] + df['entidade_mundofisi']).head(2)

0    42328990.0
1    43605089.0
Name: pedido_id, dtype: float64


0    0
1    0
dtype: int64

In [31]:
# Criando flag para status de pedidos aprovados.
# Notem que se o resultado da operação for uma lista, tupla ou np.array, 
# ele é automaticamente convertido para pd.Series

df['flag_apv'] = [1 if s == 18 else 0 for s in df['pedido_status_id']]

### Método s.map()

In [34]:
# Criando flag para status de pedidos suspeitos:
def x(x):
    return 1 if x == 10 else 0

lambda x: 1 if x == 10 else 0

df['flag_sus'] = df['pedido_status_id'].map(lambda x: 1 if x == 10 else 0)

In [35]:
# Usando .map() como de-para:

print(df['entidade_emissores'].value_counts())

df['entidade_emissores_str'] = df['entidade_emissores'].map({0: 'nao', 1: 'sim'})

print('\n')
print(df['entidade_emissores_str'].value_counts())

0    88697
1    11303
Name: entidade_emissores, dtype: int64


nao    88697
sim    11303
Name: entidade_emissores_str, dtype: int64


### Método df.apply()

In [37]:
ents = ['entidade_emissores','entidade_santander','entidade_mundofisi']

# axis=0 -> coluna (default)
# axis=1 -> linhas 

print(df[ents].head(10).apply('sum', axis=0), '\n')
print(df[ents].head(10).sum(axis=0), '\n')

df[ents].tail(10).apply('sum', axis=1)

entidade_emissores    0
entidade_santander    0
entidade_mundofisi    0
dtype: int64 

entidade_emissores    0
entidade_santander    0
entidade_mundofisi    0
dtype: int64 



99990    0
99991    0
99992    0
99993    0
99994    0
99995    0
99996    0
99997    0
99998    0
99999    0
dtype: int64

In [38]:
# Funções matemáticas básicas operam sobre mesma ideia do df.apply():

print(df[ents].head().max(axis=1), '\n')
df[ents].max(axis=0)

0    0
1    0
2    0
3    0
4    0
dtype: int64 



entidade_emissores    1
entidade_santander    1
entidade_mundofisi    1
dtype: int64

### Alternativas para case when

In [41]:
def case(x):
    if (x['pedido_status_id'] in [9,18]) & (x['entidade_santander']==1):
        return 'apv__santander'
    elif (x['pedido_status_id'] in [9,18]) & (x['entidade_santander']==0):
        return 'apv__fora_santander'
    else:
        return 'else'

df.apply(case, axis=1).value_counts()

apv__fora_santander    51860
else                   44652
apv__santander          3488
dtype: int64

### dummies

In [43]:
pd.get_dummies(df['pedido_status_id']).head()

,9.0,10.0,11.0,14.0,18.0,20.0,22.0,24.0,26.0,27.0,32.0,33.0,34.0,38.0,41.0,43.0,44.0,45.0,46.0,47.0,100.0,101.0,102.0,103.0,104.0,106.0,110.0,111.0,112.0,115.0,116.0,117.0,118.0,120.0,121.0,122.0,123.0,124.0,125.0,126.0,129.0,132.0,134.0,135.0,139.0,140.0,143.0,145.0,155.0,163.0,164.0,165.0,166.0,167.0,170.0,180.0,182.0,185.0,1000.0,11000.0,13001.0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### .str e .dt

In [48]:
df['pedido_registro'].dt.strftime('%Y-%m').head()

0    2011-08
1    2011-08
2    2011-09
3    2011-09
4    2011-10
Name: pedido_registro, dtype: object

In [53]:
cpfint = df['base'].astype(np.int64)

In [56]:
cpfint.astype(str).str.zfill(11).head()

0    09449309485
1    09449309485
2    09449309485
3    09449309485
4    09449309485
Name: base, dtype: object

## Trabalhando com índices

Ponto positivo para eficiência de filtros e joins. Os principais métodos para gerir os índices de um DF ou Series são:

* .set_index()
* .reset_index()
* .sort_index()
* .reindex()

In [201]:
minidf = (df
          .set_index(['id','pedido_id'])
          .sort_index()
          .loc[:,['pedido_registro','pedido_status_id']]
          .head(100))
minidf

pedido_registro  pedido_status_id
id        pedido_id                                          
729015091 42329990  2011-08-02 20:02:42.070              18.0
          43606089  2011-08-15 19:59:36.960              26.0
          54691557  2011-09-19 20:10:24.570              44.0
          55095321  2011-09-25 12:56:30.487              26.0
          71166777  2011-10-11 10:16:14.497              18.0
          72220616  2011-10-24 17:23:34.060              27.0
          74102981  2011-11-14 17:01:00.097              18.0
          74255434  2011-11-16 11:16:19.600              18.0
          74774177  2011-11-21 21:50:52.640              18.0
          75113642  2011-11-25 05:55:20.897              18.0
          75821226  2011-12-01 18:06:48.847              18.0
          76078766  2011-12-04 14:54:47.523              27.0
          78054050  2011-12-20 14:41:36.403              18.0
          78843136  2011-12-29 00:05:30.927              18.0
          79091354  2012-01-01 10:56:14.220              18.0
          79180861  2012-01-02 14:55:23.900              18.0
          80644461  2012-01-14 22:15:50.583              18.0
          81522751  2012-01-23 20:50:50.777              18.0
          82720306  2012-02-04 16:54:57.277              18.0
          85718219  2012-03-05 21:36:49.803               9.0
          86357960  2012-03-11 21:07:39.000               9.0
          90246124  2012-04-12 05:47:00.783              45.0
          93971322  2012-05-09 20:33:34.483              26.0
          96164419  2012-05-29 21:49:42.377              26.0
          99844306  2012-06-27 14:09:50.543              27.0
          100384944 2012-07-01 14:25:48.697              44.0
          100789355 2012-07-04 11:45:38.790              27.0
          101648543 2012-07-10 19:53:57.907              18.0
          101649659 2012-07-10 20:02:27.140              18.0
          101651700 2012-07-10 20:15:13.577              18.0
...                                     ...               ...
          304081302 2014-11-28 18:12:36.510              27.0
          304533259 2014-11-29 03:38:59.400              27.0
          321964077 2015-02-14 15:41:39.957              18.0
          340965500 2015-05-04 00:46:51.963              27.0
          354925108 2015-07-02 23:27:19.900              27.0
          354925110 2015-07-02 23:27:19.923              27.0
          354925149 2015-07-02 23:27:35.020              27.0
          355677428 2015-07-06 14:22:28.343              27.0
          428853252 2016-03-15 11:06:25.923              27.0
          459987629 2016-06-23 00:56:11.733              27.0
          479241261 2016-08-21 17:01:07.680              27.0
          553755533 2016-12-11 01:51:25.660              18.0
          612708254 2017-04-22 17:36:47.020              18.0
          695167568 2017-10-28 06:21:57.953              27.0
          710392281 2017-11-24 20:26:37.600              26.0
729034207 632299445 2017-06-06 16:10:00.253              26.0
          820986520 2017-08-12 10:58:08.000               9.0
729083740 317167066 2015-01-22 13:58:39.020              18.0
          606017996 2017-04-04 16:13:23.333              18.0
          606018901 2017-04-04 16:15:02.787              18.0
729090130 457211918 2016-06-14 06:21:24.427             101.0
          669427415 2017-08-30 20:41:59.430              18.0
          686412736 2017-10-08 13:16:09.103              44.0
729113778 21464638  2010-04-22 10:58:55.857              27.0
          30980785  2011-01-13 14:29:29.107              27.0
729120880 729117489 2018-01-01 19:27:16.737              18.0
729132136 417958026 2016-02-08 15:43:10.797              18.0
          674787485 2017-09-12 14:48:53.800              47.0
729146203 608155191 2017-04-10 09:29:45.867              18.0
729147938 200816559 2013-10-24 15:23:10.277              18.0

[100 rows x 2 columns]

In [202]:
minidf.reset_index(drop=False)

,id,pedido_id,pedido_registro,pedido_status_id
0,729015091,42329990,2011-08-02 20:02:42.070,18.0
1,729015091,43606089,2011-08-15 19:59:36.960,26.0
2,729015091,54691557,2011-09-19 20:10:24.570,44.0
3,729015091,55095321,2011-09-25 12:56:30.487,26.0
4,729015091,71166777,2011-10-11 10:16:14.497,18.0
5,729015091,72220616,2011-10-24 17:23:34.060,27.0
6,729015091,74102981,2011-11-14 17:01:00.097,18.0
7,729015091,74255434,2011-11-16 11:16:19.600,18.0
8,729015091,74774177,2011-11-21 21:50:52.640,18.0
9,729015091,75113642,2011-11-25 05:55:20.897,18.0


# Manipulação de tabelas

## Funções de concatenação

* pd.concat
* pd.DataFrame.append

In [57]:
a = pd.Series([1,2,3], index=[10,9,8], name='col1')
b = pd.Series([4,5,6], name='col2')

display(a)
display(b)

10    1
9     2
8     3
Name: col1, dtype: int64

0    4
1    5
2    6
Name: col2, dtype: int64

In [219]:
# Concatenando verticalmente (equivalentes: np.vstack(), rbind do R)
pd.concat([a,b])

10    1
9     2
8     3
0     4
1     5
2     6
dtype: int64

In [62]:
# Concatenando horizontalmente (equivalentes: np.hstack(), cbind do R)
pd.concat([a,b], axis=0, ignore_index=True)

0    1
1    2
2    3
3    4
4    5
5    6
dtype: int64

In [63]:
# Concatenando horizontalmente (equivalentes: np.hstack(), cbind do R)
pd.concat([x.reset_index(drop=True) for x in [a,b]], axis=1)

,col1,col2
0,1,4
1,2,5
2,3,6


## Joins

* df.join(df2, left_on=[''], right_on=[''], how='left/inner/full/right')
* df.merge(df2,how='inner/full/left/right')
* pd.merge(df1, df2, how='inner/full/left/right')

In [ ]:
# rolling join: pd.merge_asof()

## Métodos úteis

pd.Series:

* s.isin()
* s.isna()
* s.notna()
* s.fillna()
* s.nlargest()
* s.nsmallest()
* s.unique()
* s.nunique()
* s.idxmax()
* s.idxmin()

<br> 

pd.DataFrame:

* df.isna()
* df.notna()
* df.fillna()
* df.drop_duplicates()
* df.sort_values()